For testing: To show the output

In [8]:
from keras.models import model_from_json
import operator
import cv2

In [18]:
#To Load the model
json_file = open("model.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
#load weights into new model
loaded_model.load_weights("model.h5")

cap = cv2.VideoCapture(0)

categories = {0: 'Excuse Me', 1: 'Good', 2: 'Hello', 3: 'Morning', 4: 'No', 5: 'Okay' , 6:'Smile', 7:'iloveyou', 8:'Thanks' , 9:'Yes'}

while True:
    _, frame = cap.read()
    #Mirror image
    frame = cv2.flip(frame, 1)
    
    #Coordinates of the ROI
    x1 = int(0.5*frame.shape[1])
    y1 = 10
    x2 = frame.shape[1]-10
    y2 = int(0.5*frame.shape[1])

    #Frame of ROI
    cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
    roi = frame[y1:y2, x1:x2]
    
    #Resizing the ROI
    roi = cv2.resize(roi, (64, 64)) 
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    test_image = cv2.Canny(roi,250,200)
    #_, test_image = cv2.threshold(roi, 120, 255, cv2.THRESH_BINARY)
    cv2.imshow("Test", test_image)

    result = loaded_model.predict(test_image.reshape(1, 64, 64, 1))
    prediction = {'Excuse Me': result[0][0], 
                  'Good': result[0][1], 
                  'Hello': result[0][2],
                  'Morning': result[0][3],
                  'No': result[0][4],
                  'Okay': result[0][5],
                  'Smile': result[0][6], 
                  'iloveyou': result[0][7], 
                  'Thanks': result[0][8],
                  'Yes': result[0][9]
    }
    
    #Sorting
    prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
    
    #Displaying the predictions
    cv2.putText(frame, prediction[0][0], (10, 120), cv2.FONT_HERSHEY_TRIPLEX, 2, (255,0,0), 2)    
    cv2.imshow("Frame", frame)
    
    #Wait for a key press to exit
    interrupt = cv2.waitKey(10)
    if interrupt & 0xFF == ord('q'): 
        break
 
#Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 43ms/step
